# Creates burndown chart for specified fixVersions

Specify the fixVersions that you'd like to estimate.

In [ ]:
fixVersions = ['1.12', '1.13']

Loading data using JIRA API (https://jira.readthedocs.io/en/master/)

Password is provided using ~/.rcnet file

In [ ]:
import pandas as pd
import numpy as np
from jira import JIRA


#store credentials in ~/.rcnet file
jira = JIRA('https://kainos-evolve.atlassian.net')
jql = 'project=VXT and fixversion in (' + ", ".join(fixVersions) +')'
jql

Above JQL will be used to download the data from Jira

In [ ]:
issuesInVersions = jira.search_issues(jql)

issues = pd.DataFrame()
issues['version'] = ''
issues['key'] = ''
issues['type'] = ''
issues['status'] = ''
issues['SP'] = 0
issues['summary'] = ''

#add issues to dataframe
for issue in issuesInVersions:
    #issue may have many versions - in this approach, one version per issue is recommended
    for fixVersion in issue.fields.fixVersions:
        if(fixVersion.name in fixVersions):
            
            if issue.fields.aggregatetimeestimate is not None:
                remainingTime = issue.fields.aggregatetimeestimate / 60 / 60 / 8 #in days
            else:
                remainingTime = np.nan
            
            issues = issues.append(
                {'version': fixVersion.name, 
                 'key': issue.key,
                 'type': issue.fields.issuetype.name,
                 'status': issue.fields.status.name,
                 'SP': issue.fields.customfield_10005,
                 'summary': issue.fields.summary,
                 'team' : str(issue.fields.customfield_14200),
                 'remainingTime' : remainingTime
                }, ignore_index=True)
            
issues.sort_values("version", inplace=True)
issues = issues.loc[~(issues['status'].isin(['Completed', 'Rejected']))]
issues = issues.loc[(issues['team'].isin(['Gdansk Team 1']))]
issues

###### All stories should have SP value

Please review items below and add SP to issues where necessary. Missing estimations my skew the results later.

In [ ]:
issues.loc[(issues['type'] == 'Story') & (issues['SP'].isnull())]


###### Use remaining time if possible
For better accuracy use remainingTime field instead of SP when possible. 
Setting 0 in SP field where remainingTime is present.

In [ ]:
issues.loc[~(issues['remainingTime'].isnull()), ['SP']] = 0
issues

In [ ]:
#replace NaN values by 0 for grouping
issues = issues.fillna(0)
#issues = issues.groupby(['version'], as_index=False)['SP'].sum()
issues = issues.groupby(['version']).agg({'SP':'sum','remainingTime':'sum'})
issues

Manually calculate and set minimum, average and maximum velocity for the team 

In [ ]:
minVelocity = 6
avgVelocity = 13
maxVelocity = 20.5

Calculate days using velocity above

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


import pandas as pd
versions = pd.DataFrame()
#add some data to play with first
#versions['version'] = ['1.12', '1.13', '1.14']
#versions['SP'] = [33, 40, 38]

#or get data directly from Jira
versions = issues

import math
# BDay is business day, not birthday...
from pandas.tseries.offsets import BDay

#calculate days for each version using velocity and adjust with remaining time
versions['minVeloDays'] = np.ceil(versions.SP * 10 / minVelocity).astype(int) + versions.remainingTime
versions['avgVeloDays'] = np.ceil(versions.SP * 10 / avgVelocity).astype(int) + versions.remainingTime
versions['maxVeloDays'] = np.ceil(versions.SP * 10 / maxVelocity).astype(int) + versions.remainingTime

#round up days
minVeloDays = math.ceil(sum(versions.minVeloDays))
avgVeloDays = math.ceil(sum(versions.avgVeloDays))
maxVeloDays = math.ceil(sum(versions.maxVeloDays))


#calculate rolling sum of days up to the version
versions['minSum'] = versions.minVeloDays.expanding(1).sum().astype(int)
versions['avgSum'] = versions.avgVeloDays.expanding(1).sum().astype(int)
versions['maxSum'] = versions.maxVeloDays.expanding(1).sum().astype(int)

#replace NaN with 0
versions = versions.fillna(0)

#add business days to today to calculate finish date
versions['finishDateMinVelo'] = 0
versions['finishDateAvgVelo'] = 0
versions['finishDateMaxVelo'] = 0


versions.finishDateMinVelo = versions.minSum.apply(lambda x: pd.to_datetime('today') + BDay(x))
versions.finishDateAvgVelo = versions.avgSum.apply(lambda x: pd.to_datetime('today') + BDay(x))
versions.finishDateMaxVelo = versions.maxSum.apply(lambda x: pd.to_datetime('today') + BDay(x))

versions['minVeloDaysToFinish'] = 0
versions['avgVeloDaysToFinish'] = 0
versions['maxVeloDaysToFinish'] = 0
versions.minVeloDaysToFinish = minVeloDays - versions.minSum
versions.avgVeloDaysToFinish = avgVeloDays - versions.avgSum
versions.maxVeloDaysToFinish = maxVeloDays - versions.maxSum

versions = versions.reset_index()

#add first row with today to create burndown chart
versions = versions.append(
        {
          'version': 'today', 
         'finishDateMinVelo': pd.to_datetime('today'),
         'finishDateMaxVelo': pd.to_datetime('today'),
         'finishDateAvgVelo': pd.to_datetime('today'),
         'minVeloDaysToFinish': minVeloDays,
         'avgVeloDaysToFinish': avgVeloDays,
         'maxVeloDaysToFinish': maxVeloDays
        }, ignore_index=True)

versions.sort_values("finishDateMinVelo", inplace=True)

versions

Plot data

In [ ]:




%pylab inline
pylab.rcParams['figure.figsize'] = (25, 20)

_ = plt.plot(versions['finishDateMinVelo'], versions['minVeloDaysToFinish'], "-r.")
_ = plt.plot(versions['finishDateAvgVelo'], versions['avgVeloDaysToFinish'], "-g.")
_ = plt.plot(versions['finishDateMaxVelo'], versions['maxVeloDaysToFinish'], "-b.")

_ = plt.xticks(rotation='vertical')

for label, x, y in zip(versions['version'], versions['finishDateMinVelo'], versions['minVeloDaysToFinish']):
    _ = plt.annotate(
        label + " " + x.strftime('%Y-%m-%d'),
        xy=(x, y), xytext=(-20, 20),
        textcoords='offset points', ha='right', va='bottom',
        bbox=dict(boxstyle='round,pad=0.5', fc='white', alpha=0.5),
        arrowprops=dict(arrowstyle = '->', connectionstyle='arc3,rad=0'))
    
    
for label, x, y in zip(versions['version'], versions['finishDateAvgVelo'], versions['avgVeloDaysToFinish']):
    _ = plt.annotate(
        label + " " + x.strftime('%Y-%m-%d'),
        xy=(x, y), xytext=(-20, 20),
        textcoords='offset points', ha='right', va='bottom',
        bbox=dict(boxstyle='round,pad=0.5', fc='white', alpha=0.5),
        arrowprops=dict(arrowstyle = '->', connectionstyle='arc3,rad=0'))


for label, x, y in zip(versions['version'], versions['finishDateMaxVelo'], versions['maxVeloDaysToFinish']):
    _ = plt.annotate(
        label + " " + x.strftime('%Y-%m-%d'),
        xy=(x, y), xytext=(-20, 20),
        textcoords='offset points', ha='right', va='bottom',
        bbox=dict(boxstyle='round,pad=0.5', fc='white', alpha=0.5),
        arrowprops=dict(arrowstyle = '->', connectionstyle='arc3,rad=0'))
